In [1]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
import pymongo

In [2]:
import sys ; sys.path.append('/home/yuting/PycharmProjects/Twinews')
print(sys.path)

['/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks', '/home/yuting/anaconda3/lib/python37.zip', '/home/yuting/anaconda3/lib/python3.7', '/home/yuting/anaconda3/lib/python3.7/lib-dynload', '', '/home/yuting/anaconda3/lib/python3.7/site-packages', '/home/yuting/PycharmProjects/Twinews', '/home/yuting/anaconda3/lib/python3.7/site-packages/shap-0.34.0-py3.7-linux-x86_64.egg', '/home/yuting/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/yuting/.ipython', '/home/yuting/PycharmProjects/Twinews']


In [3]:
from systemtools.printer import bp
from twinews.utils import *

In [4]:
import pandas as pd
import json

In [5]:
ConnUsers = getUsersCollection()

twinews users (version 1.0) initialised.


In [119]:
ConnNews = getNewsCollection()

twinews news (version 1.0) initialised.


# json dump

In [9]:
def readMongo(con,myquery):
    output = []
    try:
        #myquery = {} # 查询条件
        for item in con.find(myquery):
            del item['_id']
            output.append(item)
        print ('data reading finished')
    except Exception as e:
        print (e)
    
    return output


In [10]:
def getJson(con, myquery,table_name):
    dataDir = '/home/yuting/PycharmProjects/data'
    data = readMongo(con,myquery)
    with open(dataDir + "/" + table_name + '.json', 'w', encoding="UTF-8") as jf:
        jf.write(json.dumps(data, indent=2))    

In [ ]:
getJson(ConnNews,{},'news')

data reading finished


In [41]:
getJson(ConnUsers,{{},{"user_id":1},{"news":1}},'users_test')

data reading finished


In [227]:
logger = Logger(tmpDir('logs') + "/dssm_yf.log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

# get training data urls

In [376]:
userHistory = {}
negativeNewList = []

# read user with more than 30 news
for row in ConnUsers.find().limit(10):
    if len(row["news"]) >= 30:
        userHistory[row["user_id"]] = row["news"]
# get news for negative sampling     
for row in ConnNews.find().limit(100):
    negativeNewList.append(row["url"])

In [377]:
dataUrls = [] # composed of user_id, 20 history as query, 10 history as positive doc, 10 from neg as negative doc

for k in userHistory.keys():
    query20 = random.sample(userHistory[k],20)
    doc10 = random.sample(list(set(userHistory[k]).difference(set(query20))),10)
    dataUrls.append([k,query20,doc10])
    
assert len(negativeNewList)//len(userHistory.keys()) >= 10   #make sure we have enough negative sample

for k in range(len(dataUrls)):
    try:
        negTemp = []
        #if len(negativeNewList) > 10:
        negTemp = random.sample(list(set(negativeNewList).difference(set(userHistory[dataUrls[k][0]]))),10)
        dataUrls[k].append(negTemp)
        negativeNewList = list(set(negativeNewList)^set(negTemp))
    except Exception as e:
        print(e)

# to csv

In [282]:
import csv
from nltk.stem import WordNetLemmatizer
from nlptools.preprocessing import *
from nlptools.basics import *

In [275]:
with open('/home/yuting/PycharmProjects/data/dssm_test', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    for row in dataUrls:
        writer.writerow(row)

# sampling and labeling

In [378]:
queryList = []
posList = []
negList = []

for i in range(len(dataUrls)):
#     queryList.append(dataUrls[i][1])
#     posList.append(dataUrls[i][2])
#     negList.append(dataUrls[i][3])
    for j in range(len(dataUrls[i][1])):
        queryList.append(dataUrls[i][1][j])
    for k in range(len(dataUrls[i][2])):
        posList.append(dataUrls[i][2][k])
    for m in range(len(dataUrls[i][3])):
        negList.append(dataUrls[i][3][m])
urlsList = queryList + posList + negList

In [379]:
sentences = getNewsText(urlsList, logger=logger)
bp(sentences, logger=logger)

  0% [                    ]
 10% [==                  ] (9.54s left)
 20% [====                ] (8.48s left)
 30% [======              ] (7.56s left)
 40% [========            ] (6.375s left)
 50% [==========          ] (5.43s left)
 60% [============        ] (4.24s left)
 70% [==============      ] (3.094s left)
 80% [================    ] (2.035s left)
 90% [==================  ] (0.993s left)
100% [====================] (total duration: 9.67s, mean duration: 0.04s)
[
  'Why do so many of us, as female artists, have to go to war? 'No to opening the door to him at all h,
  It was yells of pure joy for one school in Puerto Rico. Interested in Hurricanes? Add Hurricanes as ,
  ...,
  MADRID, March __int_2__ Xinhua) The Amancio Ortega Foundation announced Wednesday that it would dona,
  HUNTINGTON (WOWK TV) printing is the future and it right here in our area. The Robert C Byrd Institu
]


In [359]:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ T, h, ..., m, . ], [ M, a, ..., t, . ], ..., [ B, L, ..., e, . ], [ (, W, ..., t, . ] ]


In [345]:
if True:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

Lower casing   0% [                    ]
Lower casing  10% [==                  ] (0.45s left)
Lower casing  20% [====                ] (0.439s left)
Lower casing  30% [======              ] (0.326s left)
Lower casing  40% [========            ] (0.285s left)
Lower casing  50% [==========          ] (0.23s left)
Lower casing  60% [============        ] (0.18s left)
Lower casing  70% [==============      ] (0.137s left)
Lower casing  80% [================    ] (0.084s left)
Lower casing  90% [==================  ] (0.039s left)
Lower casing 100% [====================] (total duration: 0.39s, mean duration: 0s)
[ [ the, weinstein, ..., them, . ], [ massive, wildfires, ..., report, . ], ..., [ blacksburg, ,, ..., here, . ], [ (, wsav, ..., it, . ] ]


In [346]:
if True:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

Lemmatization   0% [                    ]
Lemmatization  10% [==                  ] (4.77s left)
Lemmatization  20% [====                ] (4.16s left)
Lemmatization  30% [======              ] (3.36s left)
Lemmatization  40% [========            ] (2.909s left)
Lemmatization  50% [==========          ] (2.35s left)
Lemmatization  60% [============        ] (1.893s left)
Lemmatization  70% [==============      ] (1.418s left)
Lemmatization  80% [================    ] (0.894s left)
Lemmatization  90% [==================  ] (0.421s left)
Lemmatization 100% [====================] (total duration: 4s, mean duration: 0.002s)
[ [ the, weinstein, ..., them, . ], [ massive, wildfire, ..., report, . ], ..., [ blacksburg, ,, ..., here, . ], [ (, wsav, ..., it, . ] ]


In [347]:
docs = filterCorpus(docs, minDF=1/2000, maxDF=300,
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

Voc removed because of maxDF (300 elements):
{ ", ', (, ), ,, -, ., :, ;, ?, ..., woman, work, working, world, would, year, yet, york, you, your }
0.75% of voc will be removed.
[ [ weinstein, co., ..., victim, feel ], [ massive, wildfire, ..., zee, contributed ], ..., [ blacksburg, va., ..., follow, link ], [ wsav, name, ..., definitely, worth ] ]


In [380]:
urlDocs = dict()
for i in range(len(urlsList)):
    urlDocs[urlsList[i]] = sentences[i]

bp(urlDocs, 2, logger)

{
  http://a.msn.com/01/en-us/BBGSv8x?ocid=st: BEN MARGOT Ben Margot Associated Press __int_4__ Some of the key Uber meetings took place at the Fou,
  http://abcn.ws/2B5RHZX: It was yells of pure joy for one school in Puerto Rico. Interested in Hurricanes? Add Hurricanes as ,
  http://abcn.ws/2EO1Gp3: Police in Florida issued arrest warrants for nine men on Tuesday in connection with the November haz,
  http://abcn.ws/2hAYPGp: Sen. Jeff Flake, a frequent sparring partner of President Donald Trump, continues to make enemies in,
  http://abcn.ws/2yfx5Rl: Massive wildfires sweeping through California have killed at least __int_2__ people and damaged thou,
  ...,
  https://www.wkbw.com/news/new-app-will-allow-users-to-request-sexual-consent-before-dating: BUFFALO, N.Y. (WKBW) Want to request sexual consent before dating? There an app for that. It called ,
  https://www.wkbw.com/sports/bills/2017-buffalo-bills-all-22-in-review-defensive-ends: (WKBW) For the Buffalo Bills and all but two fra

In [381]:
h = ["sentence1","sentence2","label"]
with open('/home/yuting/PycharmProjects/data/dssm_test_dev.csv', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow(h)
    for i in range(len(dataUrls)):
        query = ""
        for j in range(len(dataUrls[i][1])):
            query += urlDocs[dataUrls[i][1][j]]
            for k in range(len(dataUrls[i][2])):
                posSamp = []
                pos = urlDocs[dataUrls[i][2][k]]
                posSamp.append(query)
                posSamp.append(pos)
                posSamp.append("1")
                writer.writerow(posSamp)
            for m in range(len(dataUrls[i][3])):
                negSamp = []
                neg = urlDocs[dataUrls[i][3][m]]
                negSamp.append(query)
                negSamp.append(neg)
                negSamp.append("0")
                writer.writerow(negSamp)
    

# extra note

In [106]:
list(set(b).difference(set(a)))

[9, 4, 5]

In [108]:
set(b)^set(a)

{2, 4, 5, 9}

In [ ]:
list(set(a) ^ set(b))()